# Datamart
This notebook aims to restrict data perimeter due to the large amount of the input data

In [1]:
import pandas as pd
import numpy as np
import os
import pyspark
import pyspark.sql.functions as F

## perimeter
Movies produced between 2010 & 2019

In [25]:
title_basics = pd.read_parquet("../data/parsed_data/imdb.db/title_basics")

In [26]:
title_perimeter = title_basics[
    (title_basics.titleType == "movie") 
    & (title_basics.startYear.between(2010, 2019))][["tconst"]]

In [27]:
title_perimeter.head()

,tconst
67693,tt0069049
67839,tt0069204
98091,tt0100275
109015,tt0111414
110072,tt0112502


In [28]:
title_perimeter.to_parquet("../data/edited_data/datamart.db/title_perimeter")

## Datamart creation
### link.name.basics.title

In [38]:
perimeter = pd.read_parquet("../data/edited_data/datamart.db/title_perimeter")
link_name_basics_titles = pd.read_parquet("../data/parsed_data/imdb.db/link_name_basics_titles")

In [39]:
link_name_basics_titles = (
    link_name_basics_titles
    .merge(perimeter, on="tconst", how="inner")
    .drop(columns=["tconst"], axis=1)
    .dropna()
    .drop_duplicates()
    .merge(link_name_basics_titles, on="nconst", how="inner")
)

In [42]:
print(link_name_basics_titles.info())
link_name_basics_titles.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4696606 entries, 0 to 4696605
Data columns (total 2 columns):
nconst    object
tconst    object
dtypes: object(2)
memory usage: 107.5+ MB
None


,nconst,tconst
0,nm1380630,tt5392738
1,nm1380630,tt0357123
2,nm1380630,tt1111843
3,nm1380630,tt6304968
4,nm1473613,tt0356985


In [44]:
link_name_basics_titles.to_parquet("../data/edited_data/datamart.db/link_name_basics_titles")

### title_historical: all the movies in which an actor who played in the movie perimeter played in

In [101]:
link_name_basics_titles = pd.read_parquet("../data/edited_data/datamart.db/link_name_basics_titles")

In [102]:
titles = link_name_basics_titles[["tconst"]].drop_duplicates()

In [103]:
title_historical.shape

(669916, 0)

In [104]:
titles.to_parquet("../data/edited_data/datamart.db/titles")

#### names

In [105]:
link_name_basics_titles = pd.read_parquet("../data/edited_data/datamart.db/link_name_basics_titles")

In [106]:
names = link_name_basics_titles[["nconst"]].drop_duplicates()

In [107]:
names.shape

(1959210, 1)

In [108]:
names.to_parquet("../data/edited_data/datamart.db/names")

#### title.basics

In [109]:
title_basics = pd.read_parquet("../data/parsed_data/imdb.db/title_basics")
titles = pd.read_parquet("../data/edited_data/datamart.db/titles")

In [110]:
title_basics = title_basics.merge(titles, on="tconst", how="inner")

In [112]:
title_basics.to_parquet("../data/edited_data/datamart.db/title_basics")

#### title.akas

In [113]:
titles = pd.read_parquet("../data/edited_data/datamart.db/titles")
title_akas = pd.read_parquet("../data/parsed_data/imdb.db/title_akas")

In [114]:
title_akas = title_akas.merge(titles.rename(columns={"tconst": "titleId"}), on="titleId", how="inner")

In [116]:
title_akas.to_parquet("../data/edited_data/datamart.db/title_akas")

#### title.ratings

In [2]:
titles = pd.read_parquet("../data/edited_data/datamart.db/titles")
title_ratings = pd.read_parquet("../data/parsed_data/imdb.db/title_ratings")

In [3]:
title_ratings = title_ratings.merge(titles, on="tconst", how="inner")

In [5]:
titles.shape, title_ratings.shape

((669916, 1), (277451, 3))

In [120]:
title_ratings.to_parquet("../data/edited_data/datamart.db/title_ratings")

#### name.basics

In [121]:
names = pd.read_parquet("../data/edited_data/datamart.db/names")
name_basics = pd.read_parquet("../data/parsed_data/imdb.db/name_basics")

In [123]:
name_basics = name_basics.merge(names, on="nconst", how="inner")

In [125]:
name_basics.to_parquet("../data/edited_data/datamart.db/name_basics")

#### title.crew 

In [126]:
titles = pd.read_parquet("../data/edited_data/datamart.db/titles")
title_crew = pd.read_parquet("../data/parsed_data/imdb.db/title_crew")

In [127]:
title_crew = title_crew.merge(titles, on="tconst", how="inner")

In [129]:
title_crew.to_parquet("../data/edited_data/datamart.db/title_crew")

#### link.title.directors

In [131]:
titles = pd.read_parquet("../data/edited_data/datamart.db/titles")
link_title_directors = pd.read_parquet("../data/parsed_data/imdb.db/link_title_directors")

In [132]:
link_title_directors = link_title_directors.merge(titles, on="tconst", how="inner")

In [134]:
link_title_directors.to_parquet("../data/edited_data/datamart.db/link_title_directors")

#### title.principals

In [135]:
titles = pd.read_parquet("../data/edited_data/datamart.db/titles")
title_principals = pd.read_parquet("../data/parsed_data/imdb.db/title_principals")

In [136]:
title_principals = title_principals.merge(titles, on="tconst", how="inner")

In [138]:
title_principals.to_parquet("../data/edited_data/datamart.db/title_principals")